# Updated End to End Pipeline for Homework 1

In this notebook, we show our revised end to end pipeline for homework 1. After debugging the Random Forest Classifier, we found some patterns in the false negative and false positives. We have changed the workflow and show the updated results in this notebook

In [1]:
import sys
sys.path.append('/u/p/m/pmartinkus/Documents/CS_838')

import HW1 as hw
from sklearn import tree, svm
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

## Generate Examples
Here, we generate the examples just like we did before. However, we have added some additional columns.
1. Since we found that we were not taking into account the fact that many words are ony capitalized because they are the first word in a sentence, we are now including another column indicating the word before the previous word. We later use this column to test if the previous word is only capitalized becuase it begins a sentence.
2. We found that sometimes the following  word is capitalized only because it is the first word in a sentence. Since we remove periods at the end of candidate names (except when they end with Jr or Sr), we need to keep track of when there is a punctuation in a candidate name

In [2]:
examples, test = hw.gen_examples(4)
examples.head()

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Label
0,There,0,0,NaN,NaN,are,False,False,0
1,There are,0,0,NaN,NaN,perhaps,False,False,0
2,There are perhaps,0,0,NaN,NaN,few,False,False,0
3,There are perhaps few,0,0,NaN,NaN,women,False,False,0
4,are,1,0,There,NaN,perhaps,False,False,0


## Pruning Step
Now that we have generated all possible candidat names (with a maximum of four words), we complete the pruning step. However, we have added some new blocking rules:

1. If there is no punctuation in the name, and the following word is capitalized.
2. If the candadate name contains a common word such as a day of the week, month, or country
3. If the previous word is capitalized, but is not the start of a sentence, in the whitelist, or contains punctuation.
4. If the following word is capitalied, but is not the start of a sentence, a day of the weekm or in the whitelist.
5. If both the following and previous words are in their respective blacklist.
6. If the name contains any words longer than 2 characters that are followed by a period
7. If the following word is jr or sr
8. If the candidate is part of the word 'White House'

Note that the number of actual names has changed because we fixed some mistakes in the raw articles.

In [3]:
print('Number of candidates generated: ' + str(len(examples)))
print('Number of actual names: ' + str(len(examples[examples['Label'] == 1])))

Number of candidates generated: 180677
Number of actual names: 1284


In [4]:
blocker = hw.Blocking()
C = blocker.block_table(examples)
C.head()

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Label
0,Monica,10,0,than,common,Lewinsky,False,False,0
1,Monica Lewinsky,10,0,than,common,and,False,False,1
2,Mona,13,0,and,Lewinsky,Charen.,False,False,0
3,Mona Charen,13,0,and,Lewinsky,"However,",False,True,1
4,However,15,0,Charen.,Mona,each,False,True,0


In [5]:
print('Number of candidates after pruning: ' + str(len(C)))

Number of candidates after pruning: 6082


In [6]:
debug = blocker.debug(examples)
correct_blocked = len(debug)
debug

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Label
0,Dave Eastburn,79,22,Major,"Military,""",told,False,False,1
1,Rex Tillerson,49,27,State,of,and,False,False,1
2,Matt Schulz,97,29,said,"breach,""",senior,False,True,1
3,Guillermo del Toro,481,38,business.,movie,for,False,True,1
4,Moon Jae-in,157,62,President,Korean,attempts,True,True,1
5,Moon Jae-in,423,62,President,Korean,said,False,True,1
6,"Martin Luther King, Jr.",200,70,cited,"problem,",teachings,True,True,1
7,Bashar al Assad,156,115,President,Syrian,regime,True,True,1
8,Klay Thompson,57,133,Warriors,the,told,False,False,1
9,Phil Bredesen,29,187,Democrat,defeating,in,False,False,1


## Generate the Feature Vectors

Next, we can create the feature vectors. We will create all of the same features as before in addition to som enew features:

1. Follow_Caps: Whether or not the following word is capitalized.
2. Prev_Period: Whether or not the previous word contains a period.
3. Prev_2_Period: Whether or not the word before the previous word contians a period.
4. Prev_2_Caps: Whether the word before the previous word is capitalized.
5. Prev_Common: Whether the previous word is in our list of common words.
6. follow_Common: Whether the following word is in our list of common words.

In [7]:
features_gen = hw.FeatureGen()
feature_vecs, features = features_gen.gen_features(C)
feature_vecs.head()

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Prev_Caps,Prev_2_Caps,...,Follow_Blacklist,Prev_Period,Prev_2_Period,Prev_Common,Prev_2_Common,Follow_Common,Length,Avg_Length,Num_Words,Label
0,Monica,10,0,than,common,Lewinsky,False,False,False,False,...,False,False,False,False,False,False,6,6.0,1,0
1,Monica Lewinsky,10,0,than,common,and,False,False,False,False,...,False,False,False,False,False,False,15,7.0,2,1
2,Mona,13,0,and,Lewinsky,Charen.,False,False,False,True,...,False,False,False,False,False,False,4,4.0,1,0
3,Mona Charen,13,0,and,Lewinsky,"However,",False,True,False,True,...,False,False,False,False,False,False,11,5.0,2,1
4,However,15,0,Charen.,Mona,each,False,True,True,True,...,False,True,False,False,False,False,7,7.0,1,0


In [8]:
len(feature_vecs)

6082

## Cross Validation of ML Models

We agian complete cross validation for each of the five model types.

In [9]:
hw.cross_val(feature_vecs, features, 'precision')

,Classifier,Run 1,Run 2,Run 3,Run 4,Run 5,Run 6,Run 7,Run 8,Run 9,Run 10,Average
0,Decision Tree,0.821053,0.677083,0.778761,0.850575,0.721649,0.750000,0.785714,0.708738,0.762376,0.726496,0.758245
1,Random Forest,0.805825,0.616822,0.778761,0.825581,0.696970,0.757282,0.769231,0.758242,0.762376,0.739496,0.751059
2,SVM,0.816092,0.855263,0.846154,0.819444,0.784810,0.842857,0.804598,0.828947,0.811765,0.879518,0.828945
3,Logistic Regression,0.802469,0.723684,0.800000,0.811594,0.757143,0.805556,0.805195,0.681159,0.708861,0.848101,0.774376
4,Linear Regression,0.824324,0.736842,0.831169,0.809524,0.796610,0.841270,0.837838,0.818182,0.794118,0.857143,0.814702


In [10]:
hw.cross_val(feature_vecs, features, 'recall')

,Classifier,Run 1,Run 2,Run 3,Run 4,Run 5,Run 6,Run 7,Run 8,Run 9,Run 10,Average
0,Decision Tree,0.600000,0.550847,0.704,0.582677,0.552,0.566176,0.604651,0.623932,0.649573,0.557823,0.599168
1,Random Forest,0.615385,0.567797,0.680,0.582677,0.536,0.566176,0.573643,0.598291,0.666667,0.598639,0.598527
2,SVM,0.546154,0.550847,0.528,0.464567,0.496,0.433824,0.542636,0.538462,0.589744,0.496599,0.518683
3,Logistic Regression,0.500000,0.466102,0.512,0.440945,0.424,0.426471,0.480620,0.401709,0.478632,0.455782,0.458626
4,Linear Regression,0.469231,0.474576,0.512,0.401575,0.376,0.389706,0.480620,0.384615,0.461538,0.448980,0.439884


We again choose the Random Forest model becuase its recall is actually above 0.60. This means we only need to add some post-processing to increase precision.

## Post-processing for the Random Forest

We will now split or training data into another training set and a test set. Using this data, we will add a post-processing step to the model.

In [11]:
train_data, test_data, train_labels, test_labels = hw.split_train_test(feature_vecs, 0.3, features)
print('Length of training set: ' + str(len(train_data)))
print('Length of test set: ' + str(len(test_data)))

Length of training set: 4257
Length of test set: 1825


In [12]:
rf = RandomForestClassifier(random_state=0)
rf.fit(train_data[features], train_labels)
predictions = rf.predict(test_data[features])
results = test_data.copy()
results['Label'] = test_labels
results['Prediction'] = predictions
precision, recall, f1 = hw.evaluate(results)
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('F1: ' + str(f1))

Precision: 0.7737704918032787
Recall: 0.6344086021505376
F1: 0.6971935007385525


In [13]:
post_results = hw.post_rules(results)
precision, recall, f1 = hw.evaluate(post_results)
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('F1: ' + str(f1))

Precision: 0.9362549800796812
Recall: 0.6317204301075269
F1: 0.754414125200642


## Running on the Test Set

Now that we have our model and post-processing rules, we will run our code on the test set to see if our model is reaching our goals.

In [14]:
test.head()

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Label
0,White,0,1,NaN,NaN,House,False,False,0
1,White House,0,1,NaN,NaN,Communications,False,False,0
2,White House Communications,0,1,NaN,NaN,Director,False,False,0
3,White House Communications Director,0,1,NaN,NaN,Hope,False,False,0
4,House,1,1,White,NaN,Communications,False,False,0


In [15]:
print('Number of candidates generated: ' + str(len(test)))
print('Number of actual names: ' + str(len(test[test['Label'] == 1])))

Number of candidates generated: 89165
Number of actual names: 648


In [16]:
blocker = hw.Blocking()
C_test = blocker.block_table(test)
C_test.head()

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Label
0,Hope,4,1,Director,Communications,Hicks,False,False,0
1,Hope Hicks,4,1,Director,Communications,is,False,False,1
2,But,27,1,matter.,the,it's,False,False,0
3,Hicks,34,1,forthcoming,how,will,False,False,1
4,When,44,1,interview.,closed-door,former,False,False,0


In [17]:
print('Number of candidates after pruning: ' + str(len(C_test)))

Number of candidates after pruning: 2971


In [18]:
debug_test = blocker.debug(test)
correct_blocked_test = len(debug_test)
debug_test

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Label
0,Nanette Barragán,34,10,Democrat,by,The,False,True,1
1,Teresa Fitzgerald,201,19,Sister,said,who,False,True,1
2,Dan Elwell,178,67,Administrator,FAA,He,False,True,1
3,Kevin de Leon,19,80,"challenger,",upstart,made,False,True,1
4,Joseph Scarnati,108,90,Tempore,Pro,argue,False,False,1
5,Frances,58,99,Uber.,to,has,False,True,1
6,Frances,185,99,"one,",day,has,False,False,1
7,Donald Trump,14,105,President,as,senior,True,True,1
8,Dan Coats,80,182,Intelligence,National,previously,False,False,1
9,Theresa May,36,249,Minister,Prime,and,False,False,1


In [19]:
features_gen = hw.FeatureGen()
feature_vecs_test, features = features_gen.gen_features(C_test)
feature_vecs_test.head()

,String,Position,Article,Previous,Previous_2,Following,Possessive,Punctuation,Prev_Caps,Prev_2_Caps,...,Follow_Blacklist,Prev_Period,Prev_2_Period,Prev_Common,Prev_2_Common,Follow_Common,Length,Avg_Length,Num_Words,Label
0,Hope,4,1,Director,Communications,Hicks,False,False,True,True,...,False,False,False,True,False,False,4,4.0,1,0
1,Hope Hicks,4,1,Director,Communications,is,False,False,True,True,...,False,False,False,True,False,False,10,4.5,2,1
2,But,27,1,matter.,the,it's,False,False,False,False,...,False,True,False,False,False,False,3,3.0,1,0
3,Hicks,34,1,forthcoming,how,will,False,False,False,False,...,False,False,False,False,False,False,5,5.0,1,1
4,When,44,1,interview.,closed-door,former,False,False,False,False,...,False,True,False,False,False,False,4,4.0,1,0


In [20]:
rf = RandomForestClassifier(random_state=0)
rf.fit(feature_vecs[features], feature_vecs['Label'])
predictions = rf.predict(feature_vecs_test[features])
results = feature_vecs_test.copy()
results['Prediction'] = predictions
precision, recall, f1 = hw.evaluate(results)
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('F1: ' + str(f1))

Precision: 0.7638888888888888
Recall: 0.6062992125984252
F1: 0.6760316066725197


In [21]:
post_results = hw.post_rules(results)
precision, recall, f1 = hw.evaluate(post_results)
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('F1: ' + str(f1))

Precision: 0.9078014184397163
Recall: 0.6047244094488189
F1: 0.7258979206049149
